In [39]:
# %%capture
# !pip install pyspellchecker
# !pip install nlpaug

In [11]:
import torch
from transformers import AutoTokenizer, AutoModel
from nlpaug.augmenter.word import SynonymAug
from spellchecker import SpellChecker

In [20]:
# Load hunspell for Russian language
spell = SpellChecker(language='ru', distance=2)

# Load the RuRoberta-large model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
if torch.cuda.is_available():
    model.cuda()

In [36]:
def correct_typos(text):
    corrected_text = []
    misspelled = spell.unknown(text.split())
    for word in text.split():
        if word not in misspelled or spell.correction(word) is None:
            corrected_text.append(word)
        else:
            corrected_text.append(spell.correction(word))
        print(corrected_text)
    return ' '.join(corrected_text)

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0]

def augment_text(text):
    aug = SynonymAug(aug_src='wordnet')
    augmented_texts = aug.augment(text)
    return augmented_texts

In [40]:
# Assume you have a list of questions with typos
questions_with_typos = [
    "Что делать, если я хочу измнить группу или специализацию во ввремя обучения?",
    """
    Мы можем перевести вас в другую группу в рамках срока обучения и дополнительных 6 месяцев сверху. Срок обучения отсчитывается с даты оплаты обучения.
Количество переводов зависит от срока программы:

Если ваш продукт предусматривает возможность выбора специализации:
12 месяцев — 1 перевод на каждый блок обучения, суммарно 3 перевода на весь срок обучения.
24 месяца — 1 перевод на каждый блок обучения, суммарно 4 перевода на весь срок обучения.
36 месяцев — 1 перевод на каждый блок обучения, суммарно 4 перевода на весь срок обучения.
Если ваш продукт не предусматривает возможность выбора специализации:
6 месяцев — 1 перевод на все время обучения.
9 месяцев — 1 перевод на все время обучения.
12 месяцев — 2 перевода на все время обучения.
Узнать о сроках обучения и специализациях можно из программы обучения на странице вашего продукта.
    """
]

In [48]:
question_embeddings = []


for question in questions_with_typos:
    corrected_question = (question) # correct_typos

    question_embedding = embed_bert_cls(corrected_question, model, tokenizer)

    augmented_questions = augment_text(corrected_question)
    augmented_embeddings = [embed_bert_cls(aug_text, model, tokenizer) for aug_text in augmented_questions]

    averaged_embedding = torch.mean(torch.stack([question_embedding] + augmented_embeddings), dim=0).cpu().numpy()
    question_embeddings.append(averaged_embedding)

In [51]:
# The (a?) dumb way
question_embeddings[0] @ question_embeddings[1]

0.55581576